In [8]:
import os
import json
import pickle
import sys
import copy
sys.path.append("/mnt/lustre/jkyang/CVPR23/openpvsg")

## Building Dataset

In [132]:
import pickle
import numpy as np

def load_pickle(filepath):
    with open(filepath, "rb") as f:
        return pickle.load(f)

class PVSGRelationDataset:
    def __init__(self, anno_file, split='train', work_dir='./work_dirs/train_save_qf_1106'):
        with open(anno_file, "r") as f:
            anno = json.load(f)

        self.video_ids = []
        for data_source in ['vidor', 'epic_kitchen', 'ego4d']:
            for video_id in anno['split'][data_source][split]:
                self.video_ids.append(video_id)
        
        self.work_dir = work_dir
        self.split = split
        self.classes = anno['objects']['thing'] + anno['objects']['stuff']
        self.relations = anno['relations']

        self.videos = {}
        for video_anno in anno['data']:
            self.videos[video_anno['video_id']] = video_anno
            
    def __len__(self):
        return len(self.video_ids)
    
    def __getitem__(self, index):
        vid = self.video_ids[index]
#         vid = 'a383d099-5eef-48b5-9d1b-5e2d97632725'
        print(vid)
        video_info = copy.deepcopy(self.videos[vid])
        relation_dict = load_pickle(os.path.join(self.work_dir, vid, 'relations.pickle'))
        
        # getting all object features
        feat_list = []
        mapping_dict = {}
        for idx, key in enumerate(relation_dict['feats']):
            feat_list.append(relation_dict['feats'][key])
            mapping_dict[key] = idx
        relation_dict['feats'] = np.array(feat_list)
        
        # getting relation info
        pair_list = []
        for relation in relation_dict['relations']:
            relation['subject_index'] = mapping_dict[relation['subject_index']]
            relation['object_index'] = mapping_dict[relation['object_index']]
            pair_list.append([relation['subject_index'], relation['object_index']])
            
        # getting pair info
        relation_dict['pairs'] = pair_list
        
        return relation_dict

## Building Model

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
from transformers import BertConfig, BertModel


class ObjectEncoder(nn.Module):
    def __init__(self, feature_dim=256, hidden_dim=1024, num_heads=8, num_layers=2):
        super(ObjectEncoder, self).__init__()
        encoder_layers = nn.TransformerEncoderLayer(d_model=feature_dim, 
                                                    nhead=num_heads, 
                                                    dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, 
                                                         num_layers=num_layers)

    def forward(self, x):
        return self.transformer_encoder(x)


class PairProposalNetwork(nn.Module):
    def __init__(self, feature_dim, hidden_dim):
        super(PairProposalNetwork, self).__init__()
        self.pair_ffn = nn.Sequential(
            nn.Linear(feature_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, encoded_subjects, encoded_objects):
        sub_tokens = encoded_subjects.max(dim=1).values
        obj_tokens = encoded_objects.max(dim=1).values
        num_objects = obj_tokens.size(0)
        pair_matrix = torch.zeros(num_objects, num_objects)

        for i in range(num_objects):
            for j in range(num_objects):
                if i != j:
                    combined_features = torch.cat([sub_tokens[i], obj_tokens[j]], dim=-1)
                    pair_matrix[i, j] = self.pair_ffn(combined_features)

        return pair_matrix

In [125]:
class TemporalTransformer(nn.Module):
    def __init__(self, input_dim=512, output_dim=512, num_relations=65, num_transformer_layers=2, reduced_dim=32):
        super(TemporalTransformer, self).__init__()
        self.num_relations = num_relations
        self.chunk_size = 512  # Define chunk size
        self.overlap = 50      # Define overlap size
        config = BertConfig(
            hidden_size=output_dim,
            num_hidden_layers=num_transformer_layers,
            num_attention_heads=8,
            intermediate_size=2048
        )

        self.bert = BertModel(config)
        self.span_head = nn.Linear(output_dim, num_relations)
        self.pred_head = nn.Linear(output_dim, num_relations)

    def forward(self, x):
        # Process with BERT
        if x.size(1) > self.chunk_size:
            sequence_output = self.process_in_chunks(x)
        else:
            sequence_output = self.bert(inputs_embeds=x).last_hidden_state
        
        # Span prediction for each frame
        span_pred = self.span_head(sequence_output)
        relation_pred = self.pred_head(sequence_output)
        relation_pred = torch.max(relation_pred, dim=1).values

        return span_pred, relation_pred
    
    def process_in_chunks(self, x):
        total_length = x.size(1)
        output = []

        for i in range(100):
            start_idx = i * (self.chunk_size - self.overlap)
            end_idx = start_idx + self.chunk_size
            chunk = x[:, start_idx:end_idx, :]
            chunk_output = self.bert(inputs_embeds=chunk).last_hidden_state
            output.append(chunk_output)
            if end_idx >= total_length: break;
            
        num_chunks = len(output)
        reassembled_output = torch.zeros(x.size(0), total_length, x.size(2))

        # Reassemble the outputs
        for i, chunk_output in enumerate(output):
            start_idx = i * (self.chunk_size - self.overlap)
            end_idx = start_idx + self.chunk_size
            reassembled_output[:, start_idx:end_idx, :] += chunk_output

            # Handle the non-overlapping part
            if i > 0: reassembled_output[:, start_idx:start_idx+self.overlap, :] /= 2

        return reassembled_output[:, :x.size(1), :] 

In [127]:
output_dim

512

## Loss Function

In [93]:
def zlpr_loss(y_true, y_pred):
    """
    https://kexue.fm/archives/7359
    """
    y_pred = (1 - 2 * y_true) * y_pred
    y_pred_neg = y_pred - y_true * 9999
    y_pred_pos = y_pred - (1 - y_true) * 9999
    zeros = torch.zeros_like(y_pred[..., :1])
    y_pred_neg = torch.cat([y_pred_neg, zeros], dim=-1)
    y_pred_pos = torch.cat([y_pred_pos, zeros], dim=-1)
    neg_loss = torch.logsumexp(y_pred_neg, dim=-1)
    pos_loss = torch.logsumexp(y_pred_pos, dim=-1)
    
    total_loss = neg_loss + pos_loss
    
     # Average loss per pair
    if y_pred.ndim == 3:  # For span_pred with [batch_size, num_frames, num_relations]
        total_loss = total_loss.mean(dim=1)  # Average across frames
        
    return total_loss.mean()

## Utils

In [103]:
def pick_top_pairs(gt_relations, pred_matrix, num_total_pairs=100):
    with torch.no_grad():
        pred_matrix_flat = pred_matrix.view(-1)
        max_pairs = min(pred_matrix_flat.size(0), num_total_pairs)
        
        # Get ground truth pairs
        gt_pairs = [(relation['subject_index'], relation['object_index']) for relation in gt_relations]
        
        # Get top 100 predicted pairs
        top_scores, top_indices = torch.topk(pred_matrix_flat, max_pairs - len(gt_pairs), sorted=True)
        num_objects = pred_matrix.size(0)
        top_pairs = [(index // num_objects, index % num_objects) for index in top_indices 
                     if index // num_objects != index % num_objects]

        # Combine top predicted pairs with ground truth pairs
        combined_pairs = gt_pairs + top_pairs

        # Remove duplicates and truncate to ensure exactly num_total_pairs
        selected_pairs = list(set(combined_pairs))

        return [[int(s.item()), int(o.item())] for s, o in selected_pairs]

In [47]:
def concatenate_sub_obj(sub_feats, obj_feats, selected_pairs):
    concatenated_feats = []

    for subject_idx, object_idx in selected_pairs:
        subject_feat = sub_feats[subject_idx]
        object_feat = obj_feats[object_idx]

        # Concatenate the features along the feature dimension
        concatenated_pair_feat = torch.cat([subject_feat, object_feat], dim=-1)
        concatenated_feats.append(concatenated_pair_feat)

    # Stack all the concatenated features to create the final output tensor
    concatenated_feats = torch.stack(concatenated_feats)

    return concatenated_feats

In [92]:
def generate_gt_matrix(gt_relations, selected_pairs, span_mat_shape):
    
    # Initialize ground truth tensors with zeros
    num_pairs, num_frames, num_relations = span_mat_shape
    gt_span = torch.zeros(num_pairs, num_frames, num_relations)
    gt_prob = torch.zeros(num_pairs, num_relations)

    # Fill in the ground truth tensors
    for relation in gt_relations:
        subject_index = relation['subject_index'].item()
        object_index = relation['object_index'].item()
        relation_index = relation['relation'].item()
        relation_span = relation['relation_span'].squeeze()

        # Find the index of the pair in selected_pairs
        pair_index = selected_pairs.index([subject_index, object_index])

        # Set the ground truth for this pair and relation
        gt_span[pair_index, :, relation_index] = relation_span
        gt_prob[pair_index, relation_index] = 1

    return gt_span, gt_prob


## Main Function

In [133]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# Assuming you have defined the classes for ObjectEncoder, PairProposalNetwork, and TemporalTransformer

# Initialize the dataset and dataloader
data_dir = '../data/'
split = 'train'
work_dir = f'../work_dirs/{split}_save_qf_1106'

pvsg_rel_dataset = PVSGRelationDataset(f"{data_dir}/pvsg.json", split, work_dir)
data_loader = DataLoader(pvsg_rel_dataset, batch_size=1, shuffle=True)  # Batch size set to 1 for simplicity

# Initialize the models
feature_dim = 256
hidden_dim = 1024
num_relations = 65
num_transformer_layers = 2
input_dim = 512
output_dim = 512
num_top_pairs = 100

subject_encoder = ObjectEncoder(feature_dim=feature_dim)
object_encoder = ObjectEncoder(feature_dim=feature_dim)

pair_proposal_model = PairProposalNetwork(feature_dim, hidden_dim)
temporal_transformer = TemporalTransformer(input_dim, output_dim, num_relations, num_transformer_layers)

# Define the optimizer
optimizer = optim.Adam(list(pair_proposal_model.parameters()) + list(temporal_transformer.parameters()), lr=0.001)

# Training loop
num_epochs = 10  # Set the number of epochs
for epoch in range(num_epochs):
    for i, relation_dict in enumerate(data_loader):
        feats = relation_dict['feats'][0].float()
        gt_relations = relation_dict['relations']
        
        # Convert the features into subjects or objects
        sub_feats = subject_encoder(feats)
        obj_feats = object_encoder(feats)
        
        # Forward pass through the Pair Proposal Network
        pred_matrix = pair_proposal_model(sub_feats, obj_feats)
        
        # Get GT matrix for pair_matrix supervision
        gt_matrix = torch.zeros_like(pred_matrix)
        for relation in gt_relations:
            gt_matrix[relation['subject_index'], relation['object_index']] = 1
        
        # Loss function for 
        loss_pair = zlpr_loss(gt_matrix.reshape(1,-1), pred_matrix.reshape(1,-1))
        
        # get top pairs and concatenate features
        selected_pairs = pick_top_pairs(gt_relations, pred_matrix, num_top_pairs)
        concatenated_feats = concatenate_sub_obj(sub_feats, obj_feats, selected_pairs)
        
        print(concatenated_feats.shape)
        # Forward pass through the Temporal Transformer
        span_pred, prob = temporal_transformer(concatenated_feats)
        
        
        # Get GT content
        gt_span, gt_prob = generate_gt_matrix(gt_relations, selected_pairs, span_pred.shape)
        
        # Compute loss for span_pred and prob
        loss_span_pred = zlpr_loss(gt_span, span_pred)
        loss_prob = zlpr_loss(gt_prob, prob)
        
        loss = loss_pair + loss_span_pred + loss_prob
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print statistics
        if i % 10 == 0:  # Print every 10 mini-batches
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {loss.item()}')


P04_07
torch.Size([41, 615, 512])
torch.Size([41, 512, 512])
torch.Size([41, 153, 512])
Epoch: 1, Batch: 1, Loss: 21.238988876342773
1110_12398010263
torch.Size([75, 692, 512])
torch.Size([75, 512, 512])
torch.Size([75, 230, 512])
6f874e09-ea55-460c-8796-0ceae180bebc
torch.Size([100, 387, 512])
1202_4699001028
torch.Size([77, 124, 512])
0001_4164158586
torch.Size([67, 180, 512])
P27_03
torch.Size([56, 490, 512])
1026_9015675343
torch.Size([83, 245, 512])
dfaa7536-3453-4eab-8d70-f1624d640060
torch.Size([100, 1580, 512])
torch.Size([100, 512, 512])
torch.Size([100, 512, 512])
torch.Size([100, 512, 512])
torch.Size([100, 194, 512])
1014_7161607459
torch.Size([100, 181, 512])
1110_3382100305
torch.Size([100, 185, 512])
45e463b2-bdd5-407a-ab5c-8a5c5534e078
torch.Size([95, 1065, 512])
torch.Size([95, 512, 512])
torch.Size([95, 512, 512])
torch.Size([95, 141, 512])
Epoch: 1, Batch: 11, Loss: 13.982443809509277
1011_7047414375
torch.Size([77, 160, 512])
1130_13514394984
torch.Size([78, 445, 51

0029_5139813648
torch.Size([75, 123, 512])
P28_13
torch.Size([23, 749, 512])
torch.Size([23, 512, 512])
torch.Size([23, 287, 512])
1110_7839815880
torch.Size([78, 178, 512])
1008_4294147314
torch.Size([71, 330, 512])
0086_2479290539
torch.Size([81, 133, 512])
1025_8686972517
torch.Size([100, 192, 512])
1107_3269139059
torch.Size([100, 451, 512])
0054_2434595182
torch.Size([72, 92, 512])
Epoch: 1, Batch: 141, Loss: 6.486072540283203
0036_5322122291
torch.Size([100, 173, 512])
1027_7054879809
torch.Size([100, 450, 512])
P28_10
torch.Size([100, 305, 512])
1021_8019115246
torch.Size([87, 184, 512])
P14_05
torch.Size([46, 356, 512])
1000_4919726862
torch.Size([83, 182, 512])
1026_3443791836
torch.Size([82, 270, 512])
0052_5243493279
torch.Size([89, 450, 512])
0026_2764832695
torch.Size([78, 155, 512])
1150_6827458013
torch.Size([84, 111, 512])
Epoch: 1, Batch: 151, Loss: 7.4986677169799805
181a2c5e-5b2c-4436-ba8d-26c7d7db4bb8
torch.Size([43, 1580, 512])
torch.Size([43, 512, 512])
torch.Size

3751590f-3a97-4024-845b-b800e5df6166
torch.Size([81, 1540, 512])
torch.Size([81, 512, 512])
torch.Size([81, 512, 512])
torch.Size([81, 512, 512])
torch.Size([81, 154, 512])
1010_6234926253
torch.Size([89, 451, 512])
0063_3195040591
torch.Size([78, 257, 512])
0069_6490763929
torch.Size([86, 80, 512])
85f21d37-73c8-42e8-bdda-1ff89dd6eb15
torch.Size([100, 1052, 512])
torch.Size([100, 512, 512])
torch.Size([100, 512, 512])
torch.Size([100, 128, 512])
1008_8843452460
torch.Size([89, 188, 512])
P13_10
torch.Size([100, 883, 512])
torch.Size([100, 512, 512])
torch.Size([100, 421, 512])
1025_5502438239
torch.Size([88, 238, 512])
0021_2446450580
torch.Size([83, 92, 512])
Epoch: 1, Batch: 281, Loss: 9.765408515930176
1005_10756878424
torch.Size([83, 234, 512])
1023_4664554429
torch.Size([81, 200, 512])
1008_3243801466
torch.Size([100, 192, 512])
b275f09c-5dd2-4e8c-97de-edc1f0c8222b
torch.Size([97, 568, 512])
torch.Size([97, 512, 512])
torch.Size([97, 106, 512])
1105_4410882051
torch.Size([100, 16

torch.Size([100, 268, 512])
P04_16
torch.Size([100, 1085, 512])
torch.Size([100, 512, 512])
torch.Size([100, 512, 512])
torch.Size([100, 161, 512])
1000_3841894188
torch.Size([100, 160, 512])
Epoch: 2, Batch: 61, Loss: 8.747894287109375
P13_10
torch.Size([100, 883, 512])
torch.Size([100, 512, 512])
torch.Size([100, 421, 512])
1100_9189821251
torch.Size([74, 223, 512])
1122_6898853360
torch.Size([81, 165, 512])
0071_10770292975
torch.Size([80, 374, 512])
97eb4a18-87f2-47f2-9a67-11fca6bdae64
torch.Size([30, 1540, 512])
torch.Size([30, 512, 512])
torch.Size([30, 512, 512])
torch.Size([30, 512, 512])
torch.Size([30, 154, 512])
1009_8440208424
torch.Size([77, 125, 512])
29501ed1-77bb-4f53-aeb2-d062d5f568a9
torch.Size([79, 647, 512])
torch.Size([79, 512, 512])
torch.Size([79, 185, 512])
1122_2791983993
torch.Size([56, 122, 512])
1130_4433013703
torch.Size([100, 123, 512])
1007_3410092329
torch.Size([89, 117, 512])
Epoch: 2, Batch: 71, Loss: 6.065655708312988
1006_2838844408
torch.Size([100, 

torch.Size([89, 387, 512])
1111_3522761604
torch.Size([100, 379, 512])
1001_5247398775
torch.Size([79, 112, 512])
1005_4247712487
torch.Size([81, 336, 512])
b275f09c-5dd2-4e8c-97de-edc1f0c8222b
torch.Size([59, 568, 512])
torch.Size([59, 512, 512])
torch.Size([59, 106, 512])
P26_36
torch.Size([34, 1049, 512])
torch.Size([34, 512, 512])
torch.Size([34, 512, 512])
torch.Size([34, 125, 512])
1019_3004044251
torch.Size([90, 112, 512])
Epoch: 2, Batch: 201, Loss: 6.534522533416748
1025_3942858808
torch.Size([84, 83, 512])
1020_6410117677
torch.Size([84, 241, 512])
0cb2dd94-afb1-4e30-a62f-724f34d81777_1
torch.Size([84, 505, 512])
1202_3485644822
torch.Size([72, 226, 512])
1202_4021620287
torch.Size([100, 360, 512])
1000_6217587003
torch.Size([56, 100, 512])
1002_4060588783
torch.Size([100, 104, 512])
P02_13
torch.Size([61, 148, 512])
ff5d68b9-8486-467d-ad87-93c31e7cdcca
torch.Size([95, 1540, 512])
torch.Size([95, 512, 512])
torch.Size([95, 512, 512])
torch.Size([95, 512, 512])
torch.Size([95,

...

1006_3858520659
torch.Size([74, 206, 512])
1100_9189821251
torch.Size([72, 223, 512])
0059_6883644517
torch.Size([79, 180, 512])
5a163ffe-970a-4f67-a9e7-2ce340eaf6b1
torch.Size([72, 1540, 512])
torch.Size([72, 512, 512])
torch.Size([72, 512, 512])
torch.Size([72, 512, 512])
torch.Size([72, 154, 512])
0053_5481024800
torch.Size([100, 77, 512])
1005_5233182010
torch.Size([85, 451, 512])
1122_6898853360
torch.Size([81, 165, 512])
0066_5155578777
torch.Size([78, 128, 512])
1021_8339742435
torch.Size([97, 202, 512])
Epoch: 9, Batch: 131, Loss: 7.77960205078125
1001_9367589814
torch.Size([84, 164, 512])
1099_2403283676
torch.Size([79, 448, 512])
1019_3392323609
torch.Size([56, 85, 512])
1105_8625477585


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



torch.Size([100, 512, 512])
torch.Size([100, 125, 512])
1203_5557751246
torch.Size([89, 450, 512])
b275f09c-5dd2-4e8c-97de-edc1f0c8222b
torch.Size([59, 568, 512])
torch.Size([59, 512, 512])
torch.Size([59, 106, 512])
1103_6593447729
torch.Size([82, 368, 512])
1024_5465483933
torch.Size([85, 118, 512])
1002_7394250236
torch.Size([78, 226, 512])
Epoch: 10, Batch: 61, Loss: 6.3472580909729
a95dcb11-e878-47c0-9a3a-c2252aae40ba
torch.Size([55, 1540, 512])
torch.Size([55, 512, 512])
torch.Size([55, 512, 512])
torch.Size([55, 512, 512])
torch.Size([55, 154, 512])
0083_2516506305
torch.Size([100, 216, 512])
1001_4184282918
torch.Size([90, 132, 512])
1020_14022514063
torch.Size([74, 367, 512])
0052_5243493279
torch.Size([89, 450, 512])
1021_8575864864
torch.Size([90, 362, 512])
1010_4489734073
torch.Size([84, 121, 512])
1010_4177405471
torch.Size([97, 129, 512])
03f2ed96-1719-427d-acf4-8bf504f1d66d
torch.Size([100, 884, 512])
torch.Size([100, 512, 512])
torch.Size([100, 422, 512])
P02_14
torch.

1026_3443791836
torch.Size([82, 270, 512])
Epoch: 10, Batch: 191, Loss: 8.204693794250488
1110_7174736482
torch.Size([86, 450, 512])
1111_3522761604
torch.Size([100, 379, 512])
0098_4867754798
torch.Size([30, 108, 512])
0028_3085751774
torch.Size([100, 137, 512])
1202_8280224963
torch.Size([100, 243, 512])
1025_8686972517
torch.Size([81, 192, 512])
P04_16
torch.Size([100, 1085, 512])
torch.Size([100, 512, 512])
torch.Size([100, 512, 512])
torch.Size([100, 161, 512])
1101_5408448203
torch.Size([81, 89, 512])
1010_3497637415
torch.Size([84, 178, 512])
f1085fc6-4c8f-4ad2-9cd9-caa9dafe1b11
torch.Size([51, 1580, 512])
torch.Size([51, 512, 512])
torch.Size([51, 512, 512])
torch.Size([51, 512, 512])
torch.Size([51, 194, 512])
Epoch: 10, Batch: 201, Loss: 12.01533031463623
1120_4442953867
torch.Size([84, 450, 512])
0020_10793023296
torch.Size([72, 178, 512])
1023_4664554429
torch.Size([81, 200, 512])
1018_5618504780
torch.Size([83, 119, 512])
0045_5028176838
torch.Size([89, 203, 512])
1021_801

torch.Size([100, 380, 512])
1bfe5ac2-cbf8-4364-8a30-60d97dd395df_3
torch.Size([63, 475, 512])
0049_4822747943
torch.Size([100, 401, 512])
1052_3163478364
torch.Size([77, 132, 512])
1200_2875226866
torch.Size([89, 78, 512])
Epoch: 10, Batch: 321, Loss: 4.738578796386719
1099_2403283676
torch.Size([100, 448, 512])
1019_5588828484
torch.Size([56, 110, 512])
1202_4295889026
torch.Size([100, 113, 512])
1105_4410882051
torch.Size([82, 163, 512])
1006_3858520659
torch.Size([74, 206, 512])
1025_2940234006
torch.Size([70, 163, 512])
0001_4164158586
torch.Size([67, 180, 512])
d62f9c1c-7d01-4350-a0fb-ec553dad8cf2
torch.Size([53, 822, 512])
torch.Size([53, 512, 512])
torch.Size([53, 360, 512])
1026_9015576885
torch.Size([95, 185, 512])
1010_3748058580
torch.Size([100, 100, 512])
Epoch: 10, Batch: 331, Loss: 9.982138633728027
1155_5810255954
torch.Size([56, 433, 512])
P13_02
torch.Size([68, 346, 512])
1006_4323033143
torch.Size([87, 95, 512])
P26_01
torch.Size([52, 399, 512])
1019_6076181626
torch.

In [134]:
print('done')

done


In [129]:
relation_dict['feats'].shape

torch.Size([1, 24, 831, 256])

In [112]:
concatenated_feats

tensor([[[-1.2752, -1.0371, -0.1798,  ...,  2.0284, -1.6752,  0.9966],
         [-0.1584, -1.0383, -0.2184,  ...,  0.2761, -1.4358,  1.6404],
         [-0.0696,  0.4673, -0.1852,  ...,  1.8834, -1.8626,  1.9729],
         ...,
         [-0.1681, -0.6597,  0.1124,  ..., -0.0616,  0.1916,  0.7103],
         [ 0.4613, -0.6945, -0.4736,  ..., -0.3347, -0.5233,  1.6642],
         [ 0.6745, -1.1264, -1.1124,  ...,  0.0863, -0.1950,  2.2225]],

        [[ 0.0027, -1.2678, -1.0203,  ...,  0.2929, -0.5916,  1.2165],
         [ 0.4714, -0.4060, -0.1109,  ...,  0.7471, -1.6320,  1.9672],
         [-0.6711,  0.4476, -0.5652,  ..., -0.2189, -1.6528,  1.5728],
         ...,
         [-0.0125, -0.5800, -0.2885,  ...,  0.0981, -0.4625,  0.9061],
         [-0.2249, -0.3714, -0.4645,  ...,  0.0613, -0.2889,  0.6219],
         [-0.1125,  0.1484, -0.3920,  ...,  0.5102, -1.2350,  1.7214]],

        [[ 0.0027, -1.2678, -1.0203,  ...,  0.0395, -0.9749,  2.3050],
         [ 0.4714, -0.4060, -0.1109,  ...,  0

In [101]:
selected_pairs

[[24, 19],
 [19, 24],
 [24, 12],
 [9, 14],
 [3, 12],
 [12, 24],
 [21, 20],
 [9, 13],
 [24, 0],
 [25, 17],
 [10, 25],
 [24, 7],
 [24, 16],
 [19, 4],
 [0, 25],
 [9, 3],
 [24, 14],
 [3, 19],
 [17, 24],
 [24, 25],
 [9, 3],
 [24, 5],
 [22, 25],
 [24, 15],
 [24, 22],
 [19, 22],
 [19, 6],
 [3, 0],
 [9, 24],
 [9, 0],
 [24, 6],
 [19, 1],
 [0, 11],
 [9, 3],
 [5, 2],
 [24, 17],
 [24, 3],
 [9, 15],
 [4, 2],
 [19, 13],
 [19, 12],
 [9, 5],
 [8, 3],
 [10, 24],
 [19, 11],
 [9, 25],
 [16, 25],
 [9, 2],
 [9, 19],
 [24, 1],
 [9, 14],
 [19, 25],
 [24, 11],
 [24, 2],
 [24, 8],
 [3, 17],
 [8, 25],
 [3, 11],
 [8, 24],
 [24, 4],
 [19, 17],
 [9, 17],
 [15, 24],
 [1, 2],
 [25, 14],
 [8, 9],
 [3, 14],
 [19, 3],
 [12, 17],
 [3, 4],
 [10, 0],
 [10, 3],
 [0, 19],
 [12, 19],
 [24, 13],
 [9, 11],
 [9, 12],
 [9, 6],
 [9, 7],
 [3, 24],
 [24, 18],
 [10, 12],
 [2, 25],
 [10, 11],
 [24, 10],
 [10, 19],
 [3, 6],
 [25, 24],
 [9, 18],
 [19, 2],
 [20, 24],
 [0, 24],
 [19, 18],
 [8, 19],
 [25, 24],
 [0, 17],
 [0, 6],
 [9, 4],
